# [Annotated Types](https://pydantic-docs.helpmanual.io/usage/types/#annotated-types)

These are not in the same order as seen on the docs wiki rather they are in alphabetical order as found at `../../docs/examples/annotated_*.py`

---

## NamedTuple

In [1]:
# %load ../../docs/examples/annotated_types_named_tuple.py
from typing import NamedTuple

from pydantic import BaseModel, ValidationError


class Point(NamedTuple):
    x: int
    y: int


class Model(BaseModel):
    p: Point


print(Model(p=('1', '2')))

try:
    Model(p=('1.3', '2'))
except ValidationError as e:
    print(e)


p=Point(x=1, y=2)
1 validation error for Model
p -> x
  value is not a valid integer (type=type_error.integer)


---

## TypedDict

> Note:
>
> This is a new feature of the python standard library as of python 3.8. Prior to
> python 3.8, it requires the `typing-extensions` package. But required and optional
> fields are properly differentiated only since python 3.9. We therefore recommend
> using `typing-extensions` with python 3.8 as well (see [`requirements.txt`](../requirements.txt) file).

In [2]:
# %load ../docs/examples/annotated_types_typed_dict.py
from typing_extensions import TypedDict

from pydantic import BaseModel, Extra, ValidationError


# `total=False` means keys are non-required
class UserIdentity(TypedDict, total=False):
    name: str
    surname: str


class User(TypedDict):
    identity: UserIdentity
    age: int


class Model(BaseModel):
    u: User

    class Config:
        extra = Extra.forbid


print(Model(u={'identity': {'name': 'Smith', 'surname': 'John'}, 'age': '37'}))

print(Model(u={'identity': {'name': None, 'surname': 'John'}, 'age': '37'}))

print(Model(u={'identity': {}, 'age': '37'}))


try:
    Model(u={'identity': {'name': ['Smith'], 'surname': 'John'}, 'age': '24'})
except ValidationError as e:
    print(e)

try:
    Model(
        u={
            'identity': {'name': 'Smith', 'surname': 'John'},
            'age': '37',
            'email': 'john.smith@me.com',
        }
    )
except ValidationError as e:
    print(e)

u={'identity': {'name': 'Smith', 'surname': 'John'}, 'age': 37}
u={'identity': {'name': None, 'surname': 'John'}, 'age': 37}
u={'identity': {}, 'age': 37}
1 validation error for Model
u -> identity -> name
  str type expected (type=type_error.str)
1 validation error for Model
u -> email
  extra fields not permitted (type=value_error.extra)


In [3]:
# imports ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import sys
import ruamel.yaml

# YAML dump ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
mdl = Model(u={'identity': {'name': 'Smith', 'surname': 'John'}, 'age': '37'})

# FIXME: dump is not working
yml = ruamel.yaml.YAML()
yml.dump(mdl.dict(), sys.stdout)

u:
  identity:
    name: Smith
    surname: John
  age: 37


In [6]:
mdl

{'name': 'Smith', 'surname': 'John'}

---

## TypedDict

---